# Project: Investigating no show rates in Brazil

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

This dataset collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment. A number of characteristics about the patient are included in each row. The primary focus and intent of this data analysis is to try and determine why the no-show rate is as high as it is.  Poverty level is high and scholarships are granted based on need. The public welfare system assists where needed.

- **My dependent variable is:** No-show rate
- **My independent variables are:** SMS_received, Neighbourhood, and Age

### Questions
<ul>
<li>What effect does SMS messaging have on no-show rates?</li>
<li>Does the neighbourhood the appointment is scheduled in play a part?</li>
<li>Is there a chance that the age of the patient has a negative effect on these rates?</li>
</ul>

### Data Dictionary
1. **PatientId:** Unique identifier for patient.  Note: 1 patient can have several appointments
2. **AppointmentId:** Unique identifier for Appointment.  
3. **Gender:**Gender of patient.  Valid choices include M for Male or F for Female
4. **ScheduledDay:** Date of appointment. Typically in the future, but could be same day as AppointmentDate
5. **AppointmentDate:** Date the appointment was scheduled
6. **Age:** Patient age at time of appointment
7. **Neighbourhood:** Neighbourhood where appointment will be held.  NOT where the patient lives
8. **Scholarship:** Boolean value to indicate whether the patient receieved a scholarship
9. **Hiptertension:** Boolean value to indicate whether the patient has a history of Hipertension
10. **Diabetes:** Boolean value to indicate whether the patient has a history of Diabetes
11. **Alcoholism:** Boolean value to indicate whether the patient has a history of Alcoholism
12. **Handicap:** Integer to indicate number of handicaps reported by patient
13. **SMS_received:** Number of SMS messages received to remind patient of upcoming appointment
14. **No-show:** Boolean Value.  No means patient showed up for appointment.  Yes means patient did NOT show for for appointment

In [1]:
# Import necessary packages to analyze, manipulate data and generate visualizations
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


<a id='wrangling'></a>
## Data Wrangling

### General Properties

In [2]:
# Initial look at data by reading it in.  Looking for any obvious anomalies, errant data
# Nothing stands out at first glance, however looking at fields that I may consider dropping to focus on certain fields

df = pd.read_csv('KaggleV2-May-2016.csv')
df.head()


,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,29872499824296,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,558997776694438,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4262962299951,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,867951213174,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8841186448183,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


### Data inspection
#### Looking at the following:
1. Will use `df.shape` to determine number of rows and columns present in the data set
2. Will use `df.describe` to gain insightful information to include (but not limited to):
    1. Count
    2. Mean
    3. STD
    4. MIN
    5. MAX

In [3]:
df.shape

(110527, 14)

`df.shape` reveals 110,527 unique appointments, with 14 columns of data for each appointment

In [4]:
df.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921800e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


`df.info` will be used to look at data types, number of entries in each field etc

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   PatientId       110527 non-null  int64 
 1   AppointmentID   110527 non-null  int64 
 2   Gender          110527 non-null  object
 3   ScheduledDay    110527 non-null  object
 4   AppointmentDay  110527 non-null  object
 5   Age             110527 non-null  int64 
 6   Neighbourhood   110527 non-null  object
 7   Scholarship     110527 non-null  int64 
 8   Hipertension    110527 non-null  int64 
 9   Diabetes        110527 non-null  int64 
 10  Alcoholism      110527 non-null  int64 
 11  Handcap         110527 non-null  int64 
 12  SMS_received    110527 non-null  int64 
 13  No-show         110527 non-null  object
dtypes: int64(9), object(5)
memory usage: 11.8+ MB


`df.info` reveals:
- Each field is required (non-null)
- Every row contains a value in every field
- The number of entries matches the values seen when using `df.shape` which leads me to believe the data is relatively clean

### Initial Observations

The PatientID and AppointmentID fields don't appear to add any value to the analysis, so I'm considering dropping them as a result.  

1. Age
    1. Std is 23.11
    2. Mean is 37.08
    3. Min shows an outlier of -1.0
2. SMS_received
    1. Only 32.1% of the population received an SMS
3. Object data type
    1. According to Pandas Cookbook, the `object` datatype is a "catch-all for columns that Pandas doesn't recognize as any other specific type".  In practice, it often means that all of the values in the column are strings.

### Data Cleaning

#### Steps taken to clean up data

- Following fields dropped for reasons listed below:
    1. PatientID - Same PatientID can be used multiple times
    2. AppointmentID - Globally unique, can't develop trends based on a number like this
    3. Alcoholism - Only 3% of the population analyzed
    4. Scholarship - Only 9.8% of the population received a scholarship
    5. Handcap - Only 2% of the population anaylzed

- Rename No-show column to noshow for clarity
- Change data type of noshow from Object to Boolean
- No need to clean data any further at this point as there are no NULL values and the data otherwise looks clean

In [6]:
#Dropping fields that I don't want to use in my analysis
df.drop(['PatientId', 'AppointmentID', 'Alcoholism', 'Scholarship','Handcap','Hipertension','Diabetes'], axis=1, inplace=True)

`df.rename` will be used to rename the No-show column so it removes the mixed case lettering and dash.  This will make it easier to work with in the future

In [7]:
df.rename(columns = {'No-show':'noshow'}, inplace=True)

In [8]:
df.Age.max()

115

In [10]:
dfn = df.convert_dtypes()

In [11]:
dfn.dtypes

Gender            string
ScheduledDay      string
AppointmentDay    string
Age                Int64
Neighbourhood     string
SMS_received       Int64
noshow            string
dtype: object

In [12]:
dfn.head()

,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,SMS_received,noshow
0,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,No
1,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,No
2,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,No
3,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,No
4,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,No


In [13]:
#dfn.groupby('SMS_received').plot(kind='bar'); - took forever to run and returned the entire dataframe

In [14]:
#dfn.groupby(['Age','noshow']).plot(kind='bar'); - took forever to run and returned the entire dataframe

In [31]:
#dfn['Age','noshow'].plot(kind='bar'); - doesn't work
#ax = dfn.plot.bar(x='Age', rot=0)  - took too long to run
#dfn.columns - works
#dfn.Gender.head() - works
dfn.groupby('Age')
plt.show();

<a id='eda'></a>
## Exploratory Data Analysis

> Now that I've trimmed and cleaned my data, I'm ready to move on to exploration. I'll be computing statistics and create visualizations with the goal of addressing the research questions that I posed in the Introduction section. I'll be looking at one variable at a time, and then follow it up by looking at relationships between variables.


In [16]:
dfn.SMS_received.value_counts()

0    75045
1    35482
Name: SMS_received, dtype: Int64

In [32]:
#myplot = df.ScheduledDay.plot
noshow_by_age = dfn.groupby('Age').sum()['noshow']
plt.plot(noshow_by_age)
plt.ylabel('noshow')
plt.xlabel('Age');


KeyError: 'noshow'

### Pandas profiling report

> This report can be used interactively within the notebook.  Alternatively, there is an additional file called profile_report.html being uploaded seperately that will allow you to interact with the report more fluidly

In [17]:
profile = df.profile_report()
profile.to_widgets()
profile.to_file("profile_report.html")

Summarize dataset:   0%|          | 0/21 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Research Question 1  (Replace this header name!)

### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!

### References
- Using Pandas and Python to Explore Your Dataset https://realpython.com/pandas-python-explore-dataset
- 10 simple tips to speed up your Python data analysis: https://thenextweb.com/syndication/2020/10/12/10-simple-python-tips-to-speed-up-your-data-analysis/?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+TheNextWeb+%28The+Next+Web+All+Stories%29